# DataFrames

This notebook illustrates the *basics* of [DataFrames.jl](https://github.com/JuliaData/DataFrames.jl). 

# Load Packages

In [1]:
using Printf, Dates, Statistics, DelimitedFiles, DataFrames
include("jlFiles/printmat.jl")

printyellow (generic function with 1 method)

# Loading Some Data with DelimitedFiles or CSV.jl

The [CSV.jl](https://github.com/JuliaData/CSV.jl) package is very powerful - and also quick for large files. However, For small files, it might be easier to use `DelimitedFiles` (which comes with the Julia distribution).

In [2]:
println("the raw contents of Data/dfData1.csv:")
println(read("Data/dfData1.csv",String))

the raw contents of Data/dfData1.csv:
Date,          id,         B,      C,   D,           E     
01/05/2019,     1,      9.998,   0.000, The,        true
01/05/2019,     2,      9.994,   0.037, highway,    true
01/05/2019,     3,     10.044,   0.092, is,         true
02/05/2019,     1,     10.061,   0.098, for,        true
02/05/2019,     2,     10.076,   0.078, gamblers,   true
02/05/2019,     3,     10.064,   0.061, Better,     false


### DelimitedFiles
To use DelimtedFiles, do as in the cell below. The subsequent cell converts the dates to proper julia dates and the substrings to independent strings.

In [3]:
(data,header) = readdlm("Data/dfData1.csv", ',', header=true)       #read in the data
df1 =  identity.(DataFrame( data, vec(strip.(header)) ))            #create DataFrame

show(df1)

6×6 DataFrame
 Row │ Date        id     B        C        D          E     
     │ SubStrin…   Int64  Float64  Float64  SubStrin…  Bool  
─────┼───────────────────────────────────────────────────────
   1 │ 01/05/2019      1    9.998    0.0     The        true
   2 │ 01/05/2019      2    9.994    0.037   highway    true
   3 │ 01/05/2019      3   10.044    0.092   is         true
   4 │ 02/05/2019      1   10.061    0.098   for        true
   5 │ 02/05/2019      2   10.076    0.078   gamblers   true
   6 │ 02/05/2019      3   10.064    0.061   Better    false

In [4]:
df1[!,:D]    = convert.(String,df1[:,:D])                              #fix some of the types
df1[!,:Date] = Date.(string.(df1[:,:Date]),"dd/mm/yyyy")

show(df1)

6×6 DataFrame
 Row │ Date        id     B        C        D          E     
     │ Date        Int64  Float64  Float64  String     Bool  
─────┼───────────────────────────────────────────────────────
   1 │ 2019-05-01      1    9.998    0.0     The        true
   2 │ 2019-05-01      2    9.994    0.037   highway    true
   3 │ 2019-05-01      3   10.044    0.092   is         true
   4 │ 2019-05-02      1   10.061    0.098   for        true
   5 │ 2019-05-02      2   10.076    0.078   gamblers   true
   6 │ 2019-05-02      3   10.064    0.061   Better    false

### CSV.jl

To use CSV.jl instead, do as in the cell below. Use `normalizenames` to get names that can be used in Julia as variables names and specify the `dateformat` used in the csv file (to convert to proper Julia dates).

In [5]:
using CSV
df1_csv = CSV.read("Data/dfData1.csv",DataFrame,normalizenames=true,dateformat="dd/mm/yyyy")
show(df1_csv)

6×6 DataFrame
 Row │ Date        id     B        C        D          E     
     │ Date        Int64  Float64  Float64  String15   Bool  
─────┼───────────────────────────────────────────────────────
   1 │ 2019-05-01      1    9.998    0.0     The        true
   2 │ 2019-05-01      2    9.994    0.037   highway    true
   3 │ 2019-05-01      3   10.044    0.092   is         true
   4 │ 2019-05-02      1   10.061    0.098   for        true
   5 │ 2019-05-02      2   10.076    0.078   gamblers   true
   6 │ 2019-05-02      3   10.064    0.061   Better    false

## Picking out Rows and Columns

In [6]:
df1[:,:B]     #or df1.B, create a vector
#df1.B

6-element Vector{Float64}:
  9.998
  9.994
 10.044
 10.061
 10.076
 10.064

In [7]:
df1[(df1.id.>=2) .& (df1.E.==true), :]                #picking out some rows

Row,Date,id,B,C,D,E
,Date,Int64,Float64,Float64,String,Bool
1,2019-05-01,2,9.994,0.037,highway,true
2,2019-05-01,3,10.044,0.092,is,true
3,2019-05-02,2,10.076,0.078,gamblers,true


In [8]:
df1b = df1[2:4,[:Date,:B]]  #creates a new df
show(df1b)

3×2 DataFrame
 Row │ Date        B       
     │ Date        Float64 
─────┼─────────────────────
   1 │ 2019-05-01    9.994
   2 │ 2019-05-01   10.044
   3 │ 2019-05-02   10.061

## Changing the DataFrame

In [9]:
df1[2:3,:C] = [100,101]               #changing some values in :C
show(df1)

6×6 DataFrame
 Row │ Date        id     B        C        D          E     
     │ Date        Int64  Float64  Float64  String     Bool  
─────┼───────────────────────────────────────────────────────
   1 │ 2019-05-01      1    9.998    0.0     The        true
   2 │ 2019-05-01      2    9.994  100.0     highway    true
   3 │ 2019-05-01      3   10.044  101.0     is         true
   4 │ 2019-05-02      1   10.061    0.098   for        true
   5 │ 2019-05-02      2   10.076    0.078   gamblers   true
   6 │ 2019-05-02      3   10.064    0.061   Better    false

In [10]:
rename!(df1,:D => :BabyBlue)          #renaming a column

Row,Date,id,B,C,BabyBlue,E
,Date,Int64,Float64,Float64,String,Bool
1,2019-05-01,1,9.998,0.0,The,true
2,2019-05-01,2,9.994,100.0,highway,true
3,2019-05-01,3,10.044,101.0,is,true
4,2019-05-02,1,10.061,0.098,for,true
5,2019-05-02,2,10.076,0.078,gamblers,true
6,2019-05-02,3,10.064,0.061,Better,false


In [11]:
df1c = select(df1,[:BabyBlue,:C])              #new df
show(df1c)

println("\n")
df1b = select(df1,Not([:BabyBlue,:C]))         #new df, select and Not: all cols except those
show(df1b)

6×2 DataFrame
 Row │ BabyBlue   C       
     │ String     Float64 
─────┼────────────────────
   1 │  The         0.0
   2 │  highway   100.0
   3 │  is        101.0
   4 │  for         0.098
   5 │  gamblers    0.078
   6 │  Better      0.061

6×4 DataFrame
 Row │ Date        id     B        E     
     │ Date        Int64  Float64  Bool  
─────┼───────────────────────────────────
   1 │ 2019-05-01      1    9.998   true
   2 │ 2019-05-01      2    9.994   true
   3 │ 2019-05-01      3   10.044   true
   4 │ 2019-05-02      1   10.061   true
   5 │ 2019-05-02      2   10.076   true
   6 │ 2019-05-02      3   10.064  false

## Converting to and from a (traditional) Julia Matrix

In [12]:
arraydata = Matrix(df1[:,[:id,:B,:E]])

6×3 Matrix{Float64}:
 1.0   9.998  1.0
 2.0   9.994  1.0
 3.0  10.044  1.0
 1.0  10.061  1.0
 2.0  10.076  1.0
 3.0  10.064  0.0

In [13]:
df_from_matrix = DataFrame(arraydata,:auto)

Row,x1,x2,x3
,Float64,Float64,Float64
1,1.0,9.998,1.0
2,2.0,9.994,1.0
3,3.0,10.044,1.0
4,1.0,10.061,1.0
5,2.0,10.076,1.0
6,3.0,10.064,0.0


## Reshuffling a DataFrame (Groupby)

We now create a group for each id. These groups can be referred to as `dataG1[key]`.

We list the keys below. For instance (id=2,) is one of the keys. This is a tuple with a single element (similar to `(x,)`) where the element is `id=2`.

In [14]:
dataG1 = groupby(df1,:id)    #grouped by id

Row,Date,id,B,C,BabyBlue,E
,Date,Int64,Float64,Float64,String,Bool
1,2019-05-01,1,9.998,0.0,The,true
2,2019-05-02,1,10.061,0.098,for,true
Row,Date,id,B,C,BabyBlue,E
,Date,Int64,Float64,Float64,String,Bool
1,2019-05-01,3,10.044,101.0,is,true
2,2019-05-02,3,10.064,0.061,Better,false


In [15]:
dataG2 = vcat(dataG1...)         #put together again, but now id=1 first, then id=2

Row,Date,id,B,C,BabyBlue,E
,Date,Int64,Float64,Float64,String,Bool
1,2019-05-01,1,9.998,0.0,The,true
2,2019-05-02,1,10.061,0.098,for,true
3,2019-05-01,2,9.994,100.0,highway,true
4,2019-05-02,2,10.076,0.078,gamblers,true
5,2019-05-01,3,10.044,101.0,is,true
6,2019-05-02,3,10.064,0.061,Better,false


In [16]:
println("The keys of the grouped DataFrame are: ")
printmat(DataFrames.GroupKeys(dataG1))

The keys of the grouped DataFrame are: 
GroupKey: (id = 1,)
GroupKey: (id = 2,)
GroupKey: (id = 3,)



In [17]:
dataG1[(id=2,)]       #the group for (id=2,), 

Row,Date,id,B,C,BabyBlue,E
,Date,Int64,Float64,Float64,String,Bool
1,2019-05-01,2,9.994,100.0,highway,true
2,2019-05-02,2,10.076,0.078,gamblers,true


## Merging DataFrames

It is possible to stack DataFrames, if they are comformable.

More generally, we can use one of the many `join` functions, for instance, `innerjoin()`.

In [18]:
df3 = [df1 DataFrame(ff=1:6)]       #stacking 2 dataframes horizontally

Row,Date,id,B,C,BabyBlue,E,ff
,Date,Int64,Float64,Float64,String,Bool,Int64
1,2019-05-01,1,9.998,0.0,The,true,1
2,2019-05-01,2,9.994,100.0,highway,true,2
3,2019-05-01,3,10.044,101.0,is,true,3
4,2019-05-02,1,10.061,0.098,for,true,4
5,2019-05-02,2,10.076,0.078,gamblers,true,5
6,2019-05-02,3,10.064,0.061,Better,false,6


In [19]:
                       #loading another DataFrame, it has another order of observations
#using CSV
#df2 = CSV.read("Data/dfData2.csv",DataFrame,normalizenames=true,dateformat="dd/mm/yyyy")

(data,header) = readdlm("Data/dfData2.csv", ',', header=true)       #read in the data
df2 =  identity.(DataFrame( data, vec(strip.(header)) ))            #create DataFrame and convert types

df2[!,:Date] = Date.(string.(df2[:,:Date]),"dd/mm/yyyy")
display(df2)

Row,Date,id,G
,Date,Int64,Int64
1,2019-05-01,1,11
2,2019-05-02,1,12
3,2019-05-01,2,21
4,2019-05-02,2,22
5,2019-05-01,3,31
6,2019-05-02,3,32


In [20]:
df3 = innerjoin(df1,df2,on=[:Date,:id])       #joining 2 DataFrames, match both :Date and :id
                                              #innerjoin() for intersection of data points 

Row,Date,id,B,C,BabyBlue,E,G
,Date,Int64,Float64,Float64,String,Bool,Int64
1,2019-05-01,1,9.998,0.0,The,true,11
2,2019-05-02,1,10.061,0.098,for,true,12
3,2019-05-01,2,9.994,100.0,highway,true,21
4,2019-05-02,2,10.076,0.078,gamblers,true,22
5,2019-05-01,3,10.044,101.0,is,true,31
6,2019-05-02,3,10.064,0.061,Better,false,32


In [21]:
df3[:,:c] .= 1              #add a constant to the DataFrame
show(df3)

6×8 DataFrame
 Row │ Date        id     B        C        BabyBlue   E      G      c     
     │ Date        Int64  Float64  Float64  String     Bool   Int64  Int64 
─────┼─────────────────────────────────────────────────────────────────────
   1 │ 2019-05-01      1    9.998    0.0     The        true     11      1
   2 │ 2019-05-02      1   10.061    0.098   for        true     12      1
   3 │ 2019-05-01      2    9.994  100.0     highway    true     21      1
   4 │ 2019-05-02      2   10.076    0.078   gamblers   true     22      1
   5 │ 2019-05-01      3   10.044  101.0     is         true     31      1
   6 │ 2019-05-02      3   10.064    0.061   Better    false     32      1

## Using the Data

In [22]:
median(df1[:,:B])

10.0525

In [23]:
combine(groupby(df1,:id),:B.=>[mean,std])  #mean and std of B for each id

Row,id,B_mean,B_std
,Int64,Float64,Float64
1,1,10.0295,0.0445477
2,2,10.035,0.0579828
3,3,10.054,0.0141421
